In [17]:
import time
import requests
import pandas as pd
import json
from tqdm import tqdm
from datetime import datetime
from multiprocessing.pool import ThreadPool
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [18]:
session = requests.Session()
retry = Retry(connect=1000, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [20]:
def start(ticker):
    url = "https://api.upbit.com/v1/trades/ticks"
    now = datetime.now().strftime("%Y-%m-%d")
    date_ranges = pd.date_range('2021-01-01', now, freq='3H')
    data_df = pd.DataFrame()
    for date in tqdm(date_ranges):
        params = {
            "market": ticker,
            "count": "200",
            "to": date,
        }
        response = session.get(url, params=params)
        response_dict = json.loads(response.text)
        data_df = pd.concat([data_df, pd.DataFrame(response_dict)])
        time.sleep(0.1)
    data_df = data_df.drop_duplicates(subset=['candle_date_time_kst'])
    datas = data_df.to_dict('records')


    def insert(data):
        insert_url = 'http://127.0.0.1:8000/upbit/trades/'
        session.post(insert_url, data=data)


    pool = ThreadPool(processes=10)
    pool.map(insert, tqdm(datas))
    pool.close()
    pool.join()

In [23]:
ticker_url = 'http://127.0.0.1:8000/upbit/ticker/'
response = session.get('http://127.0.0.1:8000/upbit/ticker/')
ticker_df = pd.DataFrame(json.loads(response.text))

In [28]:
ticker_df

,market,korean_name,english_name
0,BTC-1INCH,1인치네트워크,1inch Network
1,BTC-AAVE,에이브,Aave
2,BTC-ADA,에이다,Ada
3,BTC-AERGO,아르고,Aergo
4,BTC-AHT,아하토큰,AhaToken
...,...,...,...
258,USDT-SC,시아코인,Siacoin
259,USDT-TRX,트론,TRON
260,USDT-TUSD,트루USD,TrueUSD
261,USDT-XRP,리플,Ripple


In [29]:
for ticker in ticker_df['market']:
    start(ticker)

BTC-1INCH
BTC-AAVE
BTC-ADA
BTC-AERGO
BTC-AHT
BTC-ALGO
BTC-ANKR
BTC-ARDR
BTC-ARK
BTC-ATOM
BTC-AUCTION
BTC-AUDIO
BTC-AVAX
BTC-AXS
BTC-BASIC
BTC-BCH
BTC-BFC
BTC-BNT
BTC-BORA
BTC-BSV
BTC-CBK
BTC-CELO
BTC-CHR
BTC-CHZ
BTC-COMP
BTC-CRO
BTC-CRV
BTC-CTC
BTC-CTSI
BTC-CVC
BTC-DAD
BTC-DAI
BTC-DAWN
BTC-DENT
BTC-DGB
BTC-DKA
BTC-DNT
BTC-DOGE
BTC-DOT
BTC-ELF
BTC-ENJ
BTC-EOS
BTC-ETH
BTC-FCT2
BTC-FIL
BTC-FLOW
BTC-FOR
BTC-FX
BTC-GLM
BTC-GO
BTC-GRS
BTC-GRT
BTC-GTC
BTC-GXC
BTC-HBD
BTC-HIVE
BTC-HUM
BTC-HUNT
BTC-IMX
BTC-IOST
BTC-IOTX
BTC-IQ
BTC-JST
BTC-KAVA
BTC-LINA
BTC-LINK
BTC-LOOM
BTC-LPT
BTC-LRC
BTC-LSK
BTC-LTC
BTC-LUNA
BTC-MANA
BTC-MARO
BTC-MASK
BTC-MATIC
BTC-MED
BTC-META
BTC-MFT
BTC-MKR
BTC-MLK
BTC-MOC
BTC-MTL
BTC-MVL
BTC-NEAR
BTC-NKN
BTC-NMR
BTC-NU
BTC-OBSR
BTC-OCEAN
BTC-OGN
BTC-OMG
BTC-ONIT
BTC-ORBS
BTC-OXT
BTC-PCI
BTC-PLA
BTC-POLY
BTC-POWR
BTC-PROM
BTC-PUNDIX
BTC-QTCON
BTC-QTUM
BTC-REP
BTC-RFR
BTC-RLC
BTC-RLY
BTC-RNDR
BTC-RSR
BTC-RVN
BTC-SAND
BTC-SBD
BTC-SC
BTC-SNX
BTC-SOL
BTC-SOLVE
BTC-SRM
BTC-SSX
